<a href="https://colab.research.google.com/github/nfleischmann/Gradient-Based-Methods-for-the-Training-of-Neural-Networks/blob/main/experiment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Asymptotic Properties of Stochastic Gradient Methods

One of the central results for stochastic gradient methods was that for a sufficiently small step size, the expected average squared norm of the gradients corresponding to the first $K$ iterates of a stochastic gradient method can be bounded as follows:
$$\mathbb{E} \left[\frac{1}{K}\sum_{k = 1}^K \lVert\nabla f(\theta_k)\rVert_2^2\right] \leq \; \alpha L M + \frac{2(f(\theta_1) - f_{inf})}{K \alpha}$$
For large $K$, the upper bound is proportional to the step size $\alpha$, and the noise of the stochastic vectors embodied by the term $M$. This relationship suggests that we can lower the expected average squared norm of the gradients by decreasing the step size and the noise term. In this experiment, we examine this relationship empirically for the training of neural networks.

In [ ]:
# Load libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

## Training Set

As the training set for the experiment, we use a downscaled and downsampled version of the famous MNIST dataset. Each input $x \in \mathbb{R}^{14 \times 14}$ is a $14 \times 14$ greyscale image that shows either the digit three or six. Respectively the output $y \in \{0,1\}$ takes on the value 0 for the digit three and 1 for the digit six. 

In [ ]:
# Load the mnist data set from the keras API
mnist = keras.datasets.mnist
(X, y), (X_test, y_test) = mnist.load_data()

# Subset the Classes 3 and 6
X = X[(y == 3) | (y == 6)]
y = y[(y == 3) | (y == 6)]

# Normalize the brightness of the inputs to the range [0,1]
X = X[..., np.newaxis]/255.0

# Scale down the pictures from 28x28 to 14x14 to reduce the computiational cost
X = tf.image.resize(X, (14,14)).numpy()
X = np.squeeze(X, axis=3)

# Change the labels to 0 for the digit 3 and 1 for the digit 6
y[y == 3] = 0
y[y == 6] = 1

# Transform the training set into the right format
y = np.asmatrix(y).transpose()
X = np.reshape(X, (-1, 196))

# Compute the training set size
n = X.shape[0]

## Neural Network

We use a neural network with $14 \times 14$ input neurons, three hidden layers with width ten, and a single output neuron. As the activation function for the hidden neurons, we use the ReLU and employ the logistic function for the output neuron.

In [ ]:
def build_model(input_shape=[14*14], n_hidden=3, width=10, seed=42):
  """ Builds a neural network model

    Args:
        input_shape (list): Shape of the input layer
          (default is [14*14])
        n_hidden (int): Number of hidden layers
          (default is 3)
        width (int): Width of the hidden layers
          (default is 10)
        seed (int): Seed that makes the inialization of the model reproducible
          (default is 42)

    Returns:
        tf.keras.model: Initialized neural network model
  """

  # Initialize Neural Network Model
  tf.random.set_seed(seed)
  model = keras.models.Sequential() 

  # Add input layer
  model.add(keras.layers.InputLayer(input_shape=input_shape))

  # Add hidden layers
  for layer in range(n_hidden): 
    model.add(keras.layers.Dense(width, activation="ReLU"))

  # Add output layer
  model.add(keras.layers.Dense(1,activation="sigmoid"))

  return model

## Determine the Initial Iterate

Since we are mainly interested in the asymptotic behavior, we want the second term in boundary $$\mathbb{E} \left[\frac{1}{K}\sum_{k = 1}^K \lVert\nabla f(\theta_k)\rVert_2^2\right] \leq \; \alpha L M + \frac{2(f(\theta_1) - f_{inf})}{K \alpha}$$ to vanish. One way to achieve this is by choosing an initial value $\theta_1$ for which the difference $f(\theta_1) - f_{\inf}$ is small. We determined such an initial value by applying gradient descent for 25,000 iterations.

In [ ]:
# Initialize neural network with 3 hidden layers with width 10
model = build_model()

# Perform 25.000 steps with gradient descent
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=0.1))
model.fit(X, y, batch_size=n, epochs=25000)

# Save the pretrained model
model.save('./pretrained_model')

## Stochastic and Mini-Batch Gradient Descent
For this experiment we examine stochastic and mini-batch gradient descent. The standard implementation of Keras generates the samples for the stochastic gradient methods by shuffeling the training set and cycling through it (as we explained in Section 4.5.3). For this reason, we provide an alternative implementation that draws the mini-batch uniformly (with replacement) from the training set and also starts from the determined initial iterate.

In [ ]:
def compute_squared_norm(grad):
  """ Compute squared euclidean norm

    Args:
        grad (list): List that contains the derivatives of the trainable parameters of 
                     the neural network

    Returns:
        float: Compute squared euclidean norm of the gradient
  """
  sum_of_squares = 0

  for g in grad:
      sum_of_squares = sum_of_squares + tf.math.reduce_sum(tf.square(g))
      
  return sum_of_squares.numpy()

In [ ]:
def mini_batch_gradient_descent(X, y, step_size, mini_batch_size, steps, seed):
  """ Implementation of mini-batch gradient descent

    Args:
        model: Keras model 
        X (np.ndarray): Inputs of the training set
        y (np.ndarray): Outputs of the training set
        step_size (int): Step size of the method
        mini_batch_size (int): Size of the used mini-batch
        steps (int): Number of steps for which mini-batch gradient descent is employed
        seed (int): Seed used to make the drawn mini-batches reproduceable

    Returns:
        float: Average over the squared gradients of the objective function 
               corresponding to the first 'step' iterates
  """
  # Load pretrained model 
  model = keras.models.load_model('./pretrained_model')

  # Define the loss function and the optimizer(which determines the update rule)
  log_loss = tf.keras.losses.BinaryCrossentropy()
  optimizer = keras.optimizers.SGD(learning_rate=step_size)

  # Determine the size of the training set
  n = X.shape[0]

  # Ensure that the samples for mini-batch gradient descent are repreducible
  rng = np.random.default_rng(seed)

  # Initialize the sum of the squared norm of the gradients
  sum = 0

  for step in range(steps):
    # Sample the minibatch uniformly from the training set (with replacement)
    mini_batch = rng.choice(n, mini_batch_size)
    X_mini_batch = X[mini_batch]
    y_mini_batch = y[mini_batch]
            
    # Compute the loss gradient for the mini-batch 
    with tf.GradientTape() as tape:
      pred = model(X_mini_batch, training=True)
      loss = log_loss(y_mini_batch, pred)
    grad = tape.gradient(loss, model.trainable_weights)

    # Perform a step of mini-batch gradient descent
    optimizer.apply_gradients(zip(grad, model.trainable_weights))

    # Compute the gradient on the full training set / the gradient of the objective function
    with tf.GradientTape() as tape:
      pred = model(X, training=True)
      loss = log_loss(y, pred)
    grad = tape.gradient(loss, model.trainable_weights)

    # Compute the squared norm of the gradient and add it to the sum
    sum = sum + compute_squared_norm(grad)
  return sum/steps

## Impact of the Step Size

To measure the impact of the step size, we computed the term $\frac{1}{20} \sum_{i = 1}^{20} \left[\frac{1}{30000}\sum_{k = 1}^{30000} \lVert\nabla f(\theta_k^{(i)})\rVert_2^2\right]$ multiple times for stochastic gradient descent. Each time we used a different step size $\alpha \in \{$0.01, 0.009, 0.008, 0.007, 0.006, 0.005, 0.004, 0.003, 0.002, 0.001$\}$.

In [ ]:
step_sizes = [0.001,0.0009,0.0008,0.0007,0.0006,0.0005,0.0004,0.0003,0.0002,0.0001]
seeds = list(range(20))

metrics_step_size = pd.DataFrame(columns=['step_size','avg_squared_norm_gradients'])
idx = 0 

for seed in seeds:
  rng = np.random.default_rng(seed)
  for step_size in step_sizes: 
    metrics_step_size.loc[idx, 'step_size'] = step_size
    metrics_step_size.loc[idx, 'avg_squared_norm_gradients'] = mini_batch_gradient_descent(X, y, 
                                                                                           step_size=step_size, 
                                                                                           mini_batch_size=1, 
                                                                                           steps=30000, 
                                                                                           seed=rng.choice(10000,1))
    idx = idx + 1

## Impact of the Noise Term M

In contrast to the step size, we can not adjust the noise term $M$ directly. Nevertheless, mini-batch gradient descent with mini-batch size $|B|$ reduces $M$ by a factor of $\frac{1}{|B|}$ compared to stochastic gradient descent. Therefore we try to measure the impact of the noise term $M$ by computing the term $\frac{1}{20} \sum_{i = 1}^{20} \left[\frac{1}{30000}\sum_{k = 1}^{30000} \lVert\nabla f(\theta_k^{(i)})\rVert_2^2\right]$ for mini-batch gradient descent with different mini-batch sizes. To be more precise, we fix the step size $\alpha =  0.001$ and employ the mini-batch sizes 1, 2, 3, 4, 5, 6, 7, 8, 9, and 10.

In [ ]:
mini_batch_sizes = [1,2,3,4,5,6,7,8,9,10]
seeds = list(range(20))

metrics_mb_size = pd.DataFrame(columns=['mini_batch_size','avg_squared_norm_gradients'])
idx = 0 

for seed in seeds:
  rng = np.random.default_rng(seed)
  for mini_batch_size in mini_batch_sizes: 
    metrics_mb_size.loc[idx, 'mini_batch_size'] = mini_batch_size
    metrics_mb_size.loc[idx, 'avg_squared_norm_gradients'] = mini_batch_gradient_descent(X, y, 
                                                                                         step_size=0.001, 
                                                                                         mini_batch_size=mini_batch_size, 
                                                                                         steps=30000, 
                                                                                         seed=rng.choice(10000,1))
    idx = idx + 1

## Figure 5.6

Sample observations from the modified MNIST training set.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Load the mnist data set from the keras API
mnist = keras.datasets.mnist
(X, y), (X_test, y_test) = mnist.load_data()

# Subset the Classes 3 and 6
X = X[(y == 3) | (y == 6)]
y = y[(y == 3) | (y == 6)]

# Normalize the brightness of the inputs to the range [0,1]
X = X[..., np.newaxis]/255.0

# Scale down the pictures from 28x28 to 14x14 to reduce the computiational cost
X = tf.image.resize(X, (14,14)).numpy()

fig = plt.figure(figsize=(10, 10))
gs = GridSpec(nrows=2, ncols=2)
ax0 = fig.add_subplot(gs[0, 0])
ax0.imshow(X[0, :, :, 0], cmap=plt.cm.gray_r)
ax1 = fig.add_subplot(gs[1, 0])
ax1.imshow(X[2, :, :, 0], cmap=plt.cm.gray_r)
ax2 = fig.add_subplot(gs[0, 1])
ax2.imshow(X[42, :, :, 0], cmap=plt.cm.gray_r)
ax3 = fig.add_subplot(gs[1, 1])
ax3.imshow(X[10, :, :, 0], cmap=plt.cm.gray_r)
plt.show()

fig = plt.figure(figsize=(10, 10))
gs = GridSpec(nrows=2, ncols=2)
ax0 = fig.add_subplot(gs[0, 0])
ax0.imshow(X[4, :, :, 0], cmap=plt.cm.gray_r)
ax1 = fig.add_subplot(gs[1, 0])
ax1.imshow(X[9, :, :, 0], cmap=plt.cm.gray_r)
ax2 = fig.add_subplot(gs[0, 1])
ax2.imshow(X[66, :, :, 0], cmap=plt.cm.gray_r)
ax3 = fig.add_subplot(gs[1, 1])
ax3.imshow(X[13, :, :, 0], cmap=plt.cm.gray_r)
plt.show()